
## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from collections import defaultdict

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data as data_utils
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso
import sklearn
from torch.utils.data.sampler import SubsetRandomSampler

import sys,os
sys.path.append(os.path.abspath("/mnt/home/yjo10/ceph/CAMELS/MIEST/utils/"))
from imp import reload 
# Change in mymodule/'
import vib_utils
reload(vib_utils)
from vib_utils import *

import warnings
warnings.filterwarnings('ignore')

# Device Config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu' # temporarily
# Fix random seeds for reproducibility
seed = 73
#torch.manual_seed(seed)
#np.random.seed(seed)

## Methods

In [2]:
def loader(field):
    if field == 'Mtot':
        fix = ""
    else:
        fix = "_"+field
    coef = np.load("/mnt/home/yjo10/ceph/CAMELS/MIEST/data/wph_nIllustrisTNG{}_for_vib_total.npy".format(fix))
    gcoef_avg = np.zeros((1000, coef.shape[1]))
    for i in range(1000):
        gcoef_avg[i,:] = coef[i*15:i*15+15,:].mean(axis=0)
    coef = np.load("/mnt/home/yjo10/ceph/CAMELS/MIEST/data/wph_nSIMBA{}_for_vib_total.npy".format(fix))
    rcoef_avg = np.zeros((1000, coef.shape[1]))
    for i in range(1000):
        rcoef_avg[i,:] = coef[i*15:i*15+15,:].mean(axis=0)
    coef = np.load("/mnt/home/yjo10/ceph/CAMELS/MIEST/data/wph_nAstrid{}_for_vib_total.npy".format(fix))
    acoef_avg = np.zeros((1000, coef.shape[1]))
    for i in range(1000):
        acoef_avg[i,:] = coef[i*15:i*15+15,:].mean(axis=0)
    coef = np.r_[gcoef_avg, rcoef_avg, acoef_avg]


    fparam = '/mnt/home/fvillaescusa/CAMELS/PUBLIC_RELEASE/CMD/2D_maps/data/params_IllustrisTNG.txt'
    gparams = np.loadtxt(fparam)
    gparams = gparams[:,:2] ## only Om and Sig8
    fparam = '/mnt/home/fvillaescusa/CAMELS/PUBLIC_RELEASE/CMD/2D_maps/data/params_SIMBA.txt'
    rparams = np.loadtxt(fparam)
    rparams = rparams[:,:2] ## only Om and Sig8
    fparam = "/mnt/home/fvillaescusa/CAMELS/Results/images_Astrid/params_LH_Astrid.txt" 
    aparams = np.loadtxt(fparam)
    aparams = aparams[:,:2] ## only Om and Sig8
    params  = np.r_[gparams, rparams, aparams]
    return coef, params

def make_dataset(coef,params):
    batch_size       = 100
    validation_split = .2
    shuffle_dataset  = True
    random_seed      = 42

    # Labelling for classification of simultions
    y_params   = torch.tensor(params,dtype=torch.float)
    y          = torch.zeros((y_params.shape[0],y_params.shape[1]+3))
    y[:,:2]    = y_params
    y[0:1000   ,2] = 1.
    y[1000:2000,3] = 1.
    y[2000:3000,4] = 1.


    X = torch.tensor(np.absolute(coef),dtype=torch.float)
    dataset      = data_utils.TensorDataset(X, y)
    dataset_size = len(dataset)
    indices      = list(range(dataset_size))
    split        = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader  = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    test_dataset      = data_utils.TensorDataset(X[val_indices], y[val_indices,:2])
    return train_loader, test_dataset

def print_score(test_dataset, vib, field):
    X, y            = test_dataset.tensors
    y_pred, y_sigma = vib(torch.tensor(X,dtype=torch.float).to(device))
    y_pred          = y_pred.cpu().detach().numpy()
    y               = y.cpu().detach().numpy()
    rele_om         = np.abs(y[:,0]-y_pred[:,0])/y[:,0]
    rele_om         = rele_om.mean()
    r2_om           = sklearn.metrics.r2_score(y[:,0],y_pred[:,0])
    rele_sig        = np.abs(y[:,1]-y_pred[:,1])/y[:,1]
    rele_sig        = rele_sig.mean()
    r2_sig          = sklearn.metrics.r2_score(y[:,1],y_pred[:,1])

    #print(rele_om)
    print("{}: Reletive Error=({:.3f}, {:.3f}), R2 score=({:.3f}, {:.3f})"\
          .format(field,rele_om,rele_sig,r2_om,r2_sig))


# Individual Fields

In [4]:
# Hyperparameters
beta   = 1e-3
input_shape  = 3105
output_shape = 2
learning_rate = 1e-3
decay_rate = 0.97
z_dim = 200
epochs = 3000
batch_size =100
fields =  ['Mtot','Mgas','Mstar','HI','ne','Vcdm','Z','T']

for field in fields:
    coef, params               = loader(field)
    train_loader, test_dataset = make_dataset(coef, params)
    vib = NDR(input_shape, output_shape,z_dim, num_models=3)
    total_loss, accuracy = train_ndr(vib, train_loader, device, epochs,batch_size,test_dataset, verbose=False)
    print_score(test_dataset,vib,field)
    

Mtot: Reletive Error=(0.058, 0.034), R2 score=(0.951, 0.912)
Mgas: Reletive Error=(0.094, 0.065), R2 score=(0.888, 0.649)
Mstar: Reletive Error=(0.163, 0.091), R2 score=(0.730, 0.357)
HI: Reletive Error=(0.050, 0.047), R2 score=(0.970, 0.828)
ne: Reletive Error=(0.091, 0.066), R2 score=(0.898, 0.670)
Vcdm: Reletive Error=(0.209, 0.102), R2 score=(0.606, 0.240)
Z: Reletive Error=(0.175, 0.084), R2 score=(0.696, 0.458)
T: Reletive Error=(0.138, 0.093), R2 score=(0.809, 0.329)


# Altogether

In [5]:
# Hyperparameters
beta   = 1e-3
input_shape  = 3105*8
output_shape = 2
learning_rate = 1e-3
decay_rate = 0.97
z_dim = 200
epochs = 3000
fields =  ['Mtot','Mgas','Mstar','HI','ne','Vcdm','Z','T']


for field in fields:
    coef, params               = loader(field)
    try:
        coefall = np.c_[coefall, coef]
    except:
        coefall = coef
print(coefall.shape)
        
train_loader, test_dataset = make_dataset(coefall, params)
vib = NDR(input_shape, output_shape,z_dim, num_models=3)
total_loss, accuracy = train_ndr(vib, train_loader, device, epochs,batch_size,test_dataset, verbose=False)
print_score(test_dataset,vib,field)
    

(3000, 24840)
T: Reletive Error=(0.035, 0.032), R2 score=(0.982, 0.926)
